In [ ]:

# !pip install numpy pandas scikit-learn matplotlib seaborn
# !pip install plotly
# !pip install nbformat

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:

import pandas as pd
import plotly.graph_objects as go
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
from sklearn import datasets
from sklearn.manifold import Isomap
import plotly.express as px 
import re
import random

In [ ]:
#loads in data set,, change file path
#if has index column independent set index_col = (column name)
#prints out all column headers for proofreading
df = pd.read_csv("C:\\Users\\HP\\Downloads\\combined_full_data.csv", index_col= False)
print(df.columns)

Index(['Song Name', 'mfccs_mean_0', 'mfccs_std_0', 'mfccs_mean_1',
       'mfccs_std_1', 'mfccs_mean_2', 'mfccs_std_2', 'mfccs_mean_3',
       'mfccs_std_3', 'mfccs_mean_4', 'mfccs_std_4', 'mfccs_mean_5',
       'mfccs_std_5', 'mfccs_mean_6', 'mfccs_std_6', 'mfccs_mean_7',
       'mfccs_std_7', 'mfccs_mean_8', 'mfccs_std_8', 'mfccs_mean_9',
       'mfccs_std_9', 'mfccs_mean_10', 'mfccs_std_10', 'mfccs_mean_11',
       'mfccs_std_11', 'mfccs_mean_12', 'mfccs_std_12', 'chroma_mean_0',
       'chroma_mean_1', 'chroma_mean_2', 'chroma_mean_3', 'chroma_mean_4',
       'chroma_mean_5', 'chroma_mean_6', 'chroma_mean_7', 'chroma_mean_8',
       'chroma_mean_9', 'chroma_mean_10', 'chroma_mean_11', 'chroma_std_0',
       'chroma_std_1', 'chroma_std_2', 'chroma_std_3', 'chroma_std_4',
       'chroma_std_5', 'chroma_std_6', 'chroma_std_7', 'chroma_std_8',
       'chroma_std_9', 'chroma_std_10', 'chroma_std_11', 'album', 'artist',
       'release_date', 'length', 'popularity', 'acousticness', 'danc

In [100]:
#dataset features that model looks at,, different weights for features set here

# X = df[['danceability', 'energy', 'key', 'mode', 'speechiness',
#        'acousticness', 'instrumentalness', 'valence', 'tempo',
#         ]]
X = df.copy()
weight = 2.0
weight2 = 3.0
weight3 = 0.1
X['valence']*=weight
X['speechiness']**2
X['acousticness']*=weight2
X['loudness']*=weight
X['energy']**-1

X = df[['acousticness', 'instrumentalness' , 'tempo', 'energy',
        'speechiness', 'valence', 'loudness',
        'key', 'mode', 'chroma_mean_0',
       'chroma_mean_1', 'chroma_mean_2', 'chroma_mean_3', 'chroma_mean_4',
       'chroma_mean_5', 'chroma_mean_6', 'chroma_mean_7', 'chroma_mean_8',
       'chroma_mean_9', 'chroma_mean_10', 'chroma_mean_11', 'chroma_std_0',
       'chroma_std_1', 'chroma_std_2', 'chroma_std_3', 'chroma_std_4',
       'chroma_std_5', 'chroma_std_6', 'chroma_std_7', 'chroma_std_8',
       'chroma_std_9', 'chroma_std_10', 'chroma_std_11', 'mfccs_mean_0',
       'mfccs_std_0', 'mfccs_mean_1', 'mfccs_std_1', 'mfccs_mean_2',
       'mfccs_std_2', 'mfccs_mean_3', 'mfccs_std_3', 'mfccs_mean_4',
       'mfccs_std_4', 'mfccs_mean_5', 'mfccs_std_5', 'mfccs_mean_6',
       'mfccs_std_6', 'mfccs_mean_7', 'mfccs_std_7', 'mfccs_mean_8',
       'mfccs_std_8', 'mfccs_mean_9', 'mfccs_std_9', 'mfccs_mean_10',
       'mfccs_std_10', 'mfccs_mean_11', 'mfccs_std_11', 'mfccs_mean_12',
       'mfccs_std_12']]

In [ ]:
#model
from sklearn.neighbors import NearestNeighbors

#k neighbors
model = NearestNeighbors(n_neighbors=50, p=3)

model.fit(X)

NearestNeighbors(n_neighbors=50, p=3)

In [ ]:
#converts link to id 

def getTrackIdfromLink():
    url = input("Enter Spotify track link: ")
    match = re.search(r"track/([a-zA-Z0-9]+)", url)
    if match:
        return match.group(1)
    else:
        print("No track ID found in the URL.")
        return None



In [ ]:
#turns id into uri
def convertToUriFormat(track_id):
    if track_id:
        return f"spotify:track:{track_id}"
    else:
        print("Invalid track ID provided.")
        return None

In [ ]:
#converts uri into id
def extractTrackId(uri):
    if uri and uri.startswith("spotify:track:"):
        return uri.replace("spotify:track:", "")
    else:
        print("Invalid URI format provided.")
        return None


In [ ]:
#Input track url, returns 5 track uri similar using model; nearest 20, but chooses 3 from closest 10, choose 2 from farthest 10
#prints out track name and artist,, uncomment other line for distance measurements
def get_song_recommendation():
    track_id = convertToUriFormat(getTrackIdfromLink())
    if track_id is None:
        print("No valid track ID found.")
        return 

    track_rows = df[df['uri'] == track_id]
    
    if not track_rows.empty:
        
        index_position = df[df['uri'] == track_id].index[0]
        
        distances, indices = model.kneighbors(X.iloc[[index_position]])
        
        track_name = df.iloc[index_position]['Song Name']
        print(f"\nSongs based on '{track_name}':\n")
        

        neighbor_indices = [idx for idx in indices[0] if idx != index_position]
        
#Comment out in final; only to test
#
        # print("20 Closest Neighbors:")
        # for i, idx in enumerate(neighbor_indices[:20], start=1):
        #     recommended_song = df.iloc[idx]
        #     distance = distances[0][list(indices[0]).index(idx)]
        #     print(f"{i}. {recommended_song['Song Name']} by {recommended_song['artist']} (Distance: {distance:.4f})")
    #

        nearest_10 = neighbor_indices[:10]
        next_10 = neighbor_indices[10:20]
        
        random_neighbors = random.sample(nearest_10, min(3, len(nearest_10)))
        random_neighbors += random.sample(next_10, min(2, len(next_10)))      
        output =[]

        print("5 Recommended Songs:")
        for i, idx in enumerate(random_neighbors, start=1):
            recommended_song = df.iloc[idx]
            distance = distances[0][list(indices[0]).index(idx)]
            #print(f"{i}. {recommended_song['Song Name']} by {recommended_song['artist']} (Distance: {distance:.4f})")
            print(f"{i}. {recommended_song['Song Name']} by {recommended_song['artist']}")
            output.append(extractTrackId(recommended_song['uri']))
        return output
    else:
        print("Sorry, that song's not in the database.")


In [ ]:
#Input track url, returns 20 track uri similar using model; nearest 40, but chooses 20 random
#prints out track name and artist,, uncomment other line for distance measurements
def get_song_recommendation_playlist():
    track_id = convertToUriFormat(getTrackIdfromLink())
    if track_id is None:
        print("No valid track ID found.")
        return 

    track_rows = df[df['uri'] == track_id]
    
    if not track_rows.empty:
        
        index_position = df[df['uri'] == track_id].index[0]
        
        distances, indices = model.kneighbors(X.iloc[[index_position]])
        
        track_name = df.iloc[index_position]['Song Name']
        print(f"\nSongs based on '{track_name}':\n")
        

        neighbor_indices = [idx for idx in indices[0] if idx != index_position]
        
#Comment out in final; only to test
#
        # print("40 Closest Neighbors:")
        # for i, idx in enumerate(neighbor_indices[:40], start=1):
        #     recommended_song = df.iloc[idx]
        #     distance = distances[0][list(indices[0]).index(idx)]
        #     print(f"{i}. {recommended_song['Song Name']} by {recommended_song['artist']} (Distance: {distance:.4f})")
    #

        nearest_40 = neighbor_indices[:40]
        
        random_neighbors = random.sample(nearest_40, min(20, len(nearest_40)))   
        output =[]

        print("20 Recommended Songs:")
        for i, idx in enumerate(random_neighbors, start=1):
            recommended_song = df.iloc[idx]
            distance = distances[0][list(indices[0]).index(idx)]
            #print(f"{i}. {recommended_song['Song Name']} by {recommended_song['artist']} (Distance: {distance:.4f})")
            print(f"{i}. {recommended_song['Song Name']} by {recommended_song['artist']}")
            output.append(extractTrackId(recommended_song['uri']))
        return output
    else:
        print("Sorry, that song's not in the database.")


In [90]:
get_song_recommendation()


Songs based on 'That’s Why (with GOLDN)':

5 Recommended Songs:
1. TELL ME by bear bear & friends
2. Oxygen by Excision
3. HVN ON EARTH (with Kodak Black) by Lil Tecca
4. Faded - ODESZA Remix by ZHU
5. TORE UP by Don Toliver


['3GY4DBauyY1mVlUlAB60wq',
 '6f0XqZKDhD5EOTziuKYyq1',
 '4CIyxiObT9G35HrSItAblR',
 '74tsW4h0WDYRpfD19gc10W',
 '1jKXjxMWlq4BhH6f9GtZbu']

In [93]:
get_song_recommendation_playlist()


Songs based on 'Worth It':

5 Recommended Songs:
1. Sideways by ILLENIUM
2. In My Mind by ILLENIUM
3. Fading Out by ATLiens
4. Already Know by Adventure Club
5. Returning To You (feat. Alison May) - Kill The Noise Remix by Seven Lions
6. Deep Down Low by Valentino Khan
7. Constellation by Far Out
8. There x2 by Slushii
9. I WARE HOUSE by JOYRYDE
10. Someone Else by Sullivan King
11. Start Again (feat. Fiora) by Seven Lions
12. After All - Habstrakt Remix by SLANDER
13. Sun Comes Up On You (feat. Amidy) by ARMNHMR
14. Returning To You (feat. Alison May) - Kill The Noise Remix by Seven Lions
15. Over Getting Over You by Said The Sky
16. Running To You by SLANDER
17. Ray Of Solar by Swedish House Mafia
18. Rock Ur World by Knock2
19. Do or Die by Subshock & Evangelos
20. thirst by DPR LIVE


['0PMNxtgeNSmPPp4aiA4T8F',
 '4nRbhflKt9BAc6YulQ22ga',
 '2h3Mfv3Aznwz0lXIjmTJr8',
 '3HI54kkKSEZO2wp9hV5tgo',
 '5E19QhVOEYGG54pNzed8Qu',
 '46rZgRYU1H5Xnv0n75OGis',
 '0aDYbKm7Cwe3rkjihsj7S6',
 '1tndktY8ZFXapGYIeHUGR2',
 '3THbCOOsUboMu83Ubd3sSR',
 '44MxUoeZg6FXDlHMrSbVjX',
 '41Umx93nsocrD8ckKBSuno',
 '5OhKV5ymsUcEUgvH5YWIX5',
 '1wqCrjOMRVOMktfDvVLsmv',
 '3fLD3P5Tg9hPElLCnymdL5',
 '1QScZdUhSzG4PmszARTR5n',
 '2LDv4GXuR2yR6n8bdBR95J',
 '5Y2n6pW4Vqr4Mzkd9V4Uk8',
 '5PkWhNfVEz2O9n8b6cQFMw',
 '3hzHvvELxNqWIH0hlaN4jZ',
 '7kyupZQftQDUUxlATGGmOe']

In [ ]:
#creates 2d interactive plot using 2 features, cannot portray more than 2d
#hover over points for info
fig = px.scatter(
    x=X['energy'], #liveness
    y=X['loudness'],  #danceability
    color_continuous_scale='viridis', 
    hover_data={'Song Name': df['Song Name']}  #, 'genre': df['genre']} 
)

#set labels here
fig.update_layout(
    title='Data Visualization (Energy vs Loudness)',
    xaxis_title='Energy',
    yaxis_title='Loudness'
)

fig.show()
fig.write_html("int_scatter.html")

In [ ]:
#created 3d interactive html plot
df_new= pd.DataFrame({
    'Energy': X['energy'],
    'Loudness': X['loudness'],
    'Valence': X['valence'],
    'Song Name' : df['Song Name'],
    #'artist genre': df['genre']
})

#set names below
fig = px.scatter_3d(df_new, x='Energy', y='Loudness', z='Valence', color='color', color_continuous_scale='viridis'
                    , hover_name = 'Song Name') #hover_data={'artist genre':True})
fig.update_layout(title='3D Visualization (Energy, Loudness, Valence)')


fig.show()
fig.write_html("3D_vis_of_data.html")